In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Mount Google Drive (optional - to save model directly to Drive)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install -q pillow numpy matplotlib

In [ ]:
# Upload dataset zip file
# Option 1: Upload manually via Files panel on left
# Option 2: Use this code
from google.colab import files
print("Please upload railway_defect_dataset.zip")
uploaded = files.upload()

In [ ]:
# Extract dataset
import zipfile
import os

# Unzip dataset
with zipfile.ZipFile('railway_defect_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Verify structure
!ls -la /content/railway_defect_dataset/train/
!ls -la /content/railway_defect_dataset/validation/

In [ ]:
# Configuration and Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json
from sklearn.utils.class_weight import compute_class_weight

# Dataset paths
DATASET_DIR = Path('/content/railway_defect_dataset')
TRAIN_DIR = DATASET_DIR / 'train'
VAL_DIR = DATASET_DIR / 'validation'

# Model configuration
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64  # Increased for GPU (was 32 on CPU)
EPOCHS = 50
LEARNING_RATE = 0.0001

# Output directory
OUTPUT_DIR = Path('/content/railway_defect_output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")

In [ ]:
# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('✅ Mixed precision enabled (2x faster training)')
print(f'   Compute dtype: {policy.compute_dtype}')
print(f'   Variable dtype: {policy.variable_dtype}')

In [ ]:
# Create data generators with augmentation
print("📦 Creating Data Generators...")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\n✅ Dataset loaded:")
print(f"   Training samples: {train_generator.samples}")
print(f"   Validation samples: {val_generator.samples}")
print(f"   Classes: {list(train_generator.class_indices.keys())}")
print(f"   Batch size: {BATCH_SIZE}")

# Save class indices
with open(OUTPUT_DIR / 'class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f, indent=2)

In [ ]:
# Calculate class weights to handle imbalanced dataset
print("⚖️  Calculating class weights...")

# Count samples per class
class_counts = {}
for class_name in train_generator.class_indices.keys():
    class_path = TRAIN_DIR / class_name
    count = len(list(class_path.glob('*.jpg'))) + len(list(class_path.glob('*.png')))
    class_counts[class_name] = count

total = sum(class_counts.values())

print("\nClass distribution:")
for class_name, count in sorted(class_counts.items(), key=lambda x: -x[1]):
    percentage = (count / total) * 100
    print(f"  {class_name:10s}: {count:4d} ({percentage:5.2f}%)")

# Compute class weights
class_names = sorted(train_generator.class_indices.keys(), key=lambda x: train_generator.class_indices[x])
class_counts_array = [class_counts[name] for name in class_names]
weights = compute_class_weight('balanced', classes=np.arange(len(class_names)), y=np.repeat(np.arange(len(class_names)), class_counts_array))
class_weight_dict = {i: w for i, w in enumerate(weights)}

print("\nCalculated class weights:")
for class_name, class_idx in sorted(train_generator.class_indices.items(), key=lambda x: x[1]):
    print(f"  {class_name:10s}: {class_weight_dict[class_idx]:.4f}")

In [ ]:
# Build VGG16 model
print("🏗️  Building VGG16 model...")

# Load pre-trained VGG16
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model
base_model.trainable = False

# Build classification head
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(train_generator.class_indices), activation='softmax', dtype='float32')  # Force float32 for stability
])

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

model.summary()
print("\n✅ Model built successfully")

In [ ]:
# Setup callbacks
checkpoint = ModelCheckpoint(
    str(OUTPUT_DIR / 'best_model_initial.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]
print("✅ Callbacks configured")

In [ ]:
# Train Phase 1: Frozen base model
print("\n" + "="*80)
print("🚀 Starting Training - Phase 1: Frozen Base Model")
print("="*80)

history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n✅ Phase 1 training completed")

In [ ]:
# Plot training history - Phase 1
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history_phase1.history['accuracy'], label='Train Accuracy')
plt.plot(history_phase1.history['val_accuracy'], label='Val Accuracy')
plt.title('Phase 1: Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(history_phase1.history['loss'], label='Train Loss')
plt.plot(history_phase1.history['val_loss'], label='Val Loss')
plt.title('Phase 1: Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(history_phase1.history['precision'], label='Train Precision')
plt.plot(history_phase1.history['recall'], label='Train Recall')
plt.title('Phase 1: Precision & Recall')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_history_phase1.png', dpi=150)
plt.show()

In [ ]:
# Fine-tuning Phase 2: Unfreeze top layers
print("\n" + "="*80)
print("🚀 Starting Training - Phase 2: Fine-tuning")
print("="*80)

# Unfreeze last 4 layers of VGG16
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

print(f"Trainable layers: {sum([1 for layer in model.layers if layer.trainable])}")

# Recompile with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE / 10),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Update checkpoint path
checkpoint_finetune = ModelCheckpoint(
    str(OUTPUT_DIR / 'best_model_finetuned.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

callbacks_finetune = [checkpoint_finetune, early_stop, reduce_lr]

# Fine-tune
history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,  # Fewer epochs for fine-tuning
    callbacks=callbacks_finetune,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n✅ Phase 2 fine-tuning completed")

In [ ]:
# Plot training history - Phase 2
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history_phase2.history['accuracy'], label='Train Accuracy')
plt.plot(history_phase2.history['val_accuracy'], label='Val Accuracy')
plt.title('Phase 2: Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(history_phase2.history['loss'], label='Train Loss')
plt.plot(history_phase2.history['val_loss'], label='Val Loss')
plt.title('Phase 2: Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(history_phase2.history['precision'], label='Train Precision')
plt.plot(history_phase2.history['recall'], label='Train Recall')
plt.title('Phase 2: Precision & Recall')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_history_phase2.png', dpi=150)
plt.show()

In [ ]:
# Evaluate final model
print("\n📊 Final Model Evaluation")
print("="*80)

# Load best model
best_model = keras.models.load_model(OUTPUT_DIR / 'best_model_finetuned.keras')

# Evaluate on validation set
val_loss, val_acc, val_precision, val_recall = best_model.evaluate(val_generator, verbose=1)

print(f"\n✅ Final Results:")
print(f"   Validation Accuracy:  {val_acc*100:.2f}%")
print(f"   Validation Loss:      {val_loss:.4f}")
print(f"   Validation Precision: {val_precision:.4f}")
print(f"   Validation Recall:    {val_recall:.4f}")
print(f"   F1-Score:             {2 * (val_precision * val_recall) / (val_precision + val_recall):.4f}")

In [ ]:
# Download trained model
print("\n📥 Download your trained model:")
from google.colab import files

# Zip the output directory
!zip -r railway_defect_output.zip /content/railway_defect_output/

# Download
files.download('railway_defect_output.zip')
print("\n✅ Model downloaded! Extract and copy best_model_finetuned.keras to your mobile_backend folder")

## Next Steps:

1. **Download** the `railway_defect_output.zip` file
2. **Extract** and copy `best_model_finetuned.keras` to your project
3. **Update** your mobile_backend to use the new model:
   ```bash
   cp best_model_finetuned.keras ~/Desktop/Code_on_track/mobile_backend/best_model.keras
   ```
4. **Test** the model with your rust images

## Performance Comparison:
- **MacBook Air M1**: ~30 hours (CPU only)
- **Google Colab T4 GPU**: ~2-3 hours ⚡️
- **Speed increase**: ~10-15x faster!